In [6]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
#include <time.h>

double potega(double a , int b )
{
      double wynik=1.0;
      for (int i = 1; i <= b; i++)
      {
          wynik = wynik * a;
      }
      return wynik; 
}

int main(int argc, char **argv)
{
    int nr_procesu;
    int liczba_procesow;
    int tag = 1;
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &nr_procesu);
    MPI_Comm_size(MPI_COMM_WORLD, &liczba_procesow);
    clock_t t;
    double ile_zajelo;

    if(nr_procesu == 0)
    {
      double lpi = 0 ,x = 0,total_time =0,t=0;

      for(int i=1; i<liczba_procesow; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        lpi +=x;
        MPI_Recv(&t, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        total_time=total_time+t;
      }
      printf("Wartosc liczby pi = %f \n", 4 * lpi);
      printf("Czas ktory zajelo wyliczanie = %f \n", total_time);
    }

    if(nr_procesu != 0)
    {
      t = clock();
      double x = potega( -1, nr_procesu - 1) / ( 2 * nr_procesu - 1); 
      printf("Wynik procesu %d = %f \t" , nr_procesu , x);
      MPI_Send(&x, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      t = clock() - t;
      ile_zajelo = ((double)t)/CLOCKS_PER_SEC;
      MPI_Send(&ile_zajelo, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      printf("Czas wykonywania sie procesu nr %d = %f\n",nr_procesu,ile_zajelo);
    }
    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 60 --allow-run-as-root a.out

Wynik procesu 17 = 0.030303 	Czas wykonywania sie procesu nr 17 = 0.000050
Wynik procesu 31 = 0.016393 	Czas wykonywania sie procesu nr 31 = 0.000093
Wynik procesu 39 = 0.012987 	Czas wykonywania sie procesu nr 39 = 0.000075
Wynik procesu 47 = 0.010753 	Czas wykonywania sie procesu nr 47 = 0.000067
Wynik procesu 53 = 0.009524 	Czas wykonywania sie procesu nr 53 = 0.000062
Wynik procesu 1 = 1.000000 	Czas wykonywania sie procesu nr 1 = 0.000051
Wynik procesu 12 = -0.043478 	Czas wykonywania sie procesu nr 12 = 0.000044
Wynik procesu 15 = 0.034483 	Czas wykonywania sie procesu nr 15 = 0.000084
Wynik procesu 16 = -0.032258 	Czas wykonywania sie procesu nr 16 = 0.000041
Wynik procesu 28 = -0.018182 	Czas wykonywania sie procesu nr 28 = 0.000045
Wynik procesu 29 = 0.017544 	Czas wykonywania sie procesu nr 29 = 0.000049
Wynik procesu 32 = -0.015873 	Czas wykonywania sie procesu nr 32 = 0.000043
Wynik procesu 50 = -0.010101 	Czas wykonywania sie procesu nr 50 = 0.000040
Wynik procesu 51 = 0.0